# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd 
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
root = ET.parse('./data/mondial_database.xml')

# Answer to Question 1: 10 Countries with the lowest infant mortality rates

In [6]:
infant_mortal = {}

for country in root.iterfind('country'):
    #for z in country.findall('name'):
        #print(z.text)
    for y in country.findall('infant_mortality'):
        #print(y.attrib,y.text)
        infant_mortal[country.find('name').text] = [y.text]

inf_df = pd.DataFrame.from_dict(infant_mortal, orient = 'index')
inf_df.columns = ['infant mortality']
inf_df['infant mortality'] = pd.to_numeric(inf_df['infant mortality'])
inf_df.sort_values(by='infant mortality').head(10)

,infant mortality
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


# Answer to Question 2: 10 Cities with the largest population (by year with highest number)

In [8]:
###10 cities with the largest population

cities = {}

for country in root.findall('country'):
    for city in country.findall('.//city'):
        for pop in city.findall('.//population'):
            cities[city.find('name').text] = [pop.attrib['year']] + [pop.text]
            
cities_df = pd.DataFrame.from_dict(cities, orient='index')
cities_df.columns = ['Year', 'Population']
cities_df.Population = pd.to_numeric(cities_df.Population)
cities_df.sort_values("Population", ascending=False).head(10)

,Year,Population
Shanghai,2010,22315474
Istanbul,2012,13710512
Mumbai,2011,12442373
Moskva,2013,11979529
Beijing,2010,11716620
São Paulo,2010,11152344
Tianjin,2010,11090314
Guangzhou,2010,11071424
Delhi,2011,11034555
Shenzhen,2010,10358381


# Building a DataFrame with Ethnic Groups by Country Population 

In [11]:
elements = []

def slice_per(source, step):
    return [source[i::step] for i in range(step)]

for country in root.findall('country'):
    country_name = country.find('name').text
    for population in country.findall('population'):
        country_pop = population.text
        
    for ethnicgroup in country.findall('ethnicgroup'):
        elements.append(country_name)
        ethnic_group = ethnicgroup.text
        elements.append(ethnic_group)
        ethnic_pct = ethnicgroup.attrib['percentage']
        elements.append(ethnic_pct)
        population = ethnicgroup
        elements.append(country_pop)

df = pd.DataFrame(slice_per(elements, 4)).transpose()
df.columns = ['Country', 'Ethnic Group', 'Percent of Population', 'Total Population']
df['Group Population'] = df[('Percent of Population')].astype(float)/100 * df[('Total Population')].astype(float)
df.head()

,Country,Ethnic Group,Percent of Population,Total Population,Group Population
0,Albania,Albanian,95,2800138,2.660131e+06
1,Albania,Greek,3,2800138,8.400414e+04
2,Greece,Greek,93,10816286,1.005915e+07
3,Macedonia,Macedonian,64.2,2059794,1.322388e+06
4,Macedonia,Albanian,25.2,2059794,5.190681e+05


# Answer to Question 3: The 10 Largest Ethnic Groups

In [12]:
df.groupby('Ethnic Group').sum().sort_values(by='Group Population', ascending=False).head(10)

,Group Population
Ethnic Group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [13]:
country_dict = {}

for country in root.findall('country'):
    country_code = country.attrib['car_code']
    for name in country.findall('name'):
        country_name = name.text
    country_dict[country_code] = country_name

# Answer to Question 4a: Country/Countries with the longest river

In [14]:
#name and country of a) longest river, b) largest lake and c) airport at highest elevation

rivers = []
countries = []
country_names = []
country_dict = {}

for country in root.findall('country'):
    country_code = country.attrib['car_code']
    for name in country.findall('name'):
        country_name = name.text
    country_dict[country_code] = country_name

for riv in root.findall('river'):
    country = riv.attrib['country'].split()
    country_names = list(filter(lambda x: country_dict[x], country))
    for country in countries:
        country_names.append(country)
    for item in country:
        country_name = country_dict[item]
        country_names.append(country_name)
    for name in riv.findall('name'):
        river_name = name.text
    for length in riv.findall('length'):
        river_length = length.text
        rivers.append([country_names, river_name, river_length])
        
rivers_df = pd.DataFrame.from_records(rivers, columns=['Country/Countries', 'River Name', 'River Length'])
rivers_df['River Length'] = pd.to_numeric(rivers_df['River Length'])
rivers_df.sort_values('River Length', ascending=False).head(1)['Country/Countries']
rivers_df.sort_values('River Length', ascending=False).head(1)

,Country/Countries,River Name,River Length
174,"[CO, BR, PE, Colombia, Brazil, Peru]",Amazonas,6448.0


# Answer to Question 4b: Country/Countries with the largest lake

In [15]:
lakes = []
countries = []
country_names = []

for lake in root.findall('lake'):
    country = lake.attrib['country'].split()
    country_names = list(filter(lambda x: country_dict[x], country))
    for country in countries:
        country_names.append(country)
    for item in country:
        country_name = country_dict[item]
        country_names.append(country_name)
    for name in lake.findall('name'):
        lake_name = name.text
    for area in lake.findall('area'):
        lake_area = area.text
        lakes.append([country_names, lake_name, lake_area])
        
lakes_df = pd.DataFrame.from_records(lakes, columns=['Country/Countries', 'Lake Name', 'Lake Area'])
lakes_df['Lake Area'] = pd.to_numeric(lakes_df['Lake Area'])
pd.set_option('display.max_colwidth', -1)
lakes_df.sort_values('Lake Area', ascending=False).head(1)

,Country/Countries,Lake Name,Lake Area
54,"[R, AZ, KAZ, IR, TM, Russia, Azerbaijan, Kazakhstan, Iran, Turkmenistan]",Caspian Sea,386400.0


# Answer to Question 4c: Country/Countries with the highest airport

In [16]:
airports = []

for air in root.findall('airport'):
    country = air.attrib['country'].split()
    country_names = list(filter(lambda x: country_dict[x], country))
    for country in countries:
        country_names.append(country)
    for item in country:
        country_name = country_dict[item]
        country_names.append(country_name)
    for name in air.findall('name'):
        airport_name = name.text
    for elev in air.findall('elevation'):
        airport_elev = elev.text
        airports.append([country_names, airport_name, airport_elev])

airports_df = pd.DataFrame.from_records(airports, columns=['Country/Countries', 'Airport Name', 'Airport Altitude'])
airports_df['Airport Altitude'] = pd.to_numeric(airports_df['Airport Altitude'])
airports_df.sort_values('Airport Altitude', ascending=False).dropna(0).head(1)

,Country/Countries,Airport Name,Airport Altitude
80,"[BOL, Bolivia]",El Alto Intl,4063.0
